## Descriptive Statistics 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformation import get_transformed_dataset

df = get_transformed_dataset()

#### Year

In [ ]:
# Plot the count of incidents per year
frequency_distribution = df["timeline.incident.year"].value_counts().reset_index()
frequency_distribution.columns = ["Year", "Count"]

plot = sns.barplot(x="Year", y="Count", data=frequency_distribution)
plot.set(
    xlabel = "Count", 
    ylabel = "Year", 
    title = "Incidents Coutn per Year",
)
plt.show()


### Frequency distribution

#### Threat Action

In [ ]:
# Calculate the frequency distribution of actions for the combined values
frequency_distribution = df["Actions"].value_counts().reset_index()
frequency_distribution.columns = ["Actions", "count"]

# Plot
plot = sns.barplot(x="count", y="Actions", data=frequency_distribution.head(20))
plot.set(
    xlabel = "Count", 
    ylabel = "Threat Action (Top 20)", 
    title = "Top 20 Threat Actions by Frequency",
)

plt.show()

#### Industry Names

In [ ]:
# Calculate the frequency distribution of industry names
frequency_distribution = df["victim.industry.name"].value_counts().reset_index()
frequency_distribution.columns = ["Industry", "count"]

# Plot
plot = sns.barplot(x="count", y="Industry", data=frequency_distribution)
plot.set(
    xlabel = "Count", 
    ylabel = "Industry Name", 
    title = "Frequency Distribution of Industry",
)

plt.show()